In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
datapath = "../data/data/v2.0/"
import csv
from collections import defaultdict
import seaborn as sn
from numpy.random import default_rng

In [2]:
# pip install sklearn-crfsuite

  Using cached https://files.pythonhosted.org/packages/25/74/5b7befa513482e6dee1f3dd68171a6c9dfc14c0eaa00f885ffeba54fe9b0/sklearn_crfsuite-0.3.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ca/80/7c0cad11bd99985cfe7c09427ee0b4f9bd6b048bd13d4ffb32c6db237dfb/tabulate-0.8.9-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [3]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
# from sklearn.metrics import make_scorer
# from sklearn.cross_validation import cross_val_score
# from sklearn.grid_search import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [2]:
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [3]:
english_dataframe = pd.read_csv(datapath+"hipe2020/en/HIPE-2022-v2.0-hipe2020-dev-en.tsv", sep = '\t', header = 0, engine="python", comment='#', quoting=csv.QUOTE_NONE)

In [4]:
misc_list = english_dataframe["MISC"].to_list()

In [5]:
sentence_idx = []
idx = 0
for item in misc_list:
    sentence_idx.append(idx)
    if "EndOfSentence" in item:
        idx += 1
english_dataframe["SENTENCE_ID"] = sentence_idx

In [6]:
labelled_sentences = set()
for i, label in enumerate(english_dataframe["NE-COARSE-LIT"].to_list()):
    if label != "O":
        labelled_sentences.add(sentence_idx[i])

In [7]:
token_dict = english_dataframe["TOKEN"].str.lower().value_counts().to_dict()
NE_dict = english_dataframe[english_dataframe["NE-COARSE-LIT"] != "O"]["TOKEN"].value_counts().to_dict()

In [8]:
feature_list = []
total_word_count = len(english_dataframe)
total = 0
for key in token_dict:
    total += token_dict[key]
    feature_list.append(key)
    if total > total_word_count * 0.5:
        break
print(total)
print(feature_list)
print(len(feature_list))

14543
[',', 'the', '.', 'of', 'and', 'to', 'a', 'â¬', 'in', 'is', '-', 'that', 'for', 'he', 'be', 'as', 'was', 'it', 'at', 'on', 'by', 'with', 'this', 'will', 'from', 'have', 'â€™', 'his', 'are', 'i', 'their', ';', 'which', 's', 'not', 'they', 'has', '*', 'â€œ', "'", 'you', 'an', 'or', 'one', 'all', 'â€�', 'we', 'been', 'but', 'no', 'who', 'than', 'our', 'were', 'other', 'county', 'â€”', 'any', 'them', 'man', 'made', 'when', 'new', 'more', 'can', 'j', 'out', 'there', 'may', 'day', 'would', 'its', ':', 'had', 'your']
75


In [9]:
post_fix_dict_3 = defaultdict(lambda:0)
post_fix_dict_2 = defaultdict(lambda:0)
post_fix_3_list = []
post_fix_2_list = []
for key in token_dict:
    if key == "endofsentence":
        continue
    if len(key) <= 2:
        continue
    count = token_dict[key]
    
    if len(key) == 3:
        post_fix_2 = key[-2:]
        post_fix_dict_2[post_fix_2] += count
        if post_fix_dict_2[post_fix_2] >= 30 and post_fix_2 not in post_fix_2_list:
            post_fix_2_list.append(post_fix_2)
        continue

    post_fix_3 = key[-3:]
    post_fix_dict_3[post_fix_3] += count
    if post_fix_dict_3[post_fix_3] >= 30 and post_fix_3 not in post_fix_3_list:
            post_fix_3_list.append(post_fix_3)
    

In [10]:
label_list =  list(pd.unique(english_dataframe["NE-COARSE-LIT"]))

In [11]:
feature_matrix = []
label_matrix = []
pos_dict = defaultdict(lambda: [])
label_dict = defaultdict(lambda: [])
pos = 0
progress = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
prev_label = 0
extra_features = 6
check = True
for idx in english_dataframe.index:
    if idx/len(english_dataframe) > progress[0]:
        print(progress[0])
        progress.pop(0)
    pos += 1
    row = english_dataframe.iloc[idx]
    token = row["TOKEN"]
    if isinstance(token, float): #null in the tsv is taken as a nan value
        token = "null"
    prev_label = label
    label = int(row["NE-COARSE-LIT"] != "O")
    if token.lower() in feature_list:
        pos_dict[token.lower() ].append(pos)
    if "EndOfSentence" not in row["MISC"]:
        label_dict[token].append((pos,label,prev_label,0))
        continue
    label_dict[token].append((pos,label,prev_label,1))
    # loop of all tokens in a sentence
    for token in label_dict:
        # loop over all ocurances of a token
        for item in label_dict[token]:
            feature_vector = np.full(shape = (len(feature_list)+extra_features), fill_value = np.inf) #differntiate between wheter a token occurs or wheter the current token is in the top 50%
            # label_vector = np.zeros(shape=(2))
            current_pos = item[0]
            
            # loop over all features present in a sentence
            for feature in pos_dict:
                closest_pos = pos_dict[feature][0] - current_pos # starting point
                
                # loop over all occurences of a feature present in a sentence
                for pos in pos_dict[feature]:
                    relative_pos = pos - current_pos
                    if abs(relative_pos) < abs(closest_pos):
                        closest_pos = relative_pos
                    else: break
                feature_vector[feature_list.index(feature)] = closest_pos
            feature_vector[-1] = item[0] # add location as feature
            feature_vector[-2] = item[3] # add wheter the item is the end of a sentence
            feature_vector[-3] = int(token[0].isupper()) #add wheter the first char is a capital letter
            feature_vector[-4] = int(any(char.isdigit() for char in token)) #add wheter the token contains a number
            

            # add post fixes
            if len(token) > 3:
                post_fix = token[-3:].lower()
                if post_fix in post_fix_3_list:
                    feature_vector[-5] = post_fix_3_list.index(post_fix) + 1
                    check = False
                # print(post_fix)
                else: 
                    feature_vector[-5] = 0
            if len(token) > 2:
                post_fix = token[-2:].lower()
                if post_fix in post_fix_2_list:
                    feature_vector[-6] = post_fix_2_list.index(post_fix) + 1
                else: 
                    feature_vector[-6] = 0
                # print(post_fix)
            # if not check:
            #     print(token)
            #     break
            feature_matrix.append(feature_vector)
            # label_vector[item[1]] = 1
            label_matrix.append(item[1])
    # break
    pos = 0
    pos_dict = defaultdict(lambda: [])
    label_dict = defaultdict(lambda: [])
    prev_label = 0
feature_matrix = np.array(feature_matrix)
label_matrix = np.array(label_matrix)


0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
